CNN으로 MNIST 분류 실습을 하자.  

### 1. 모델 이해하기

우리가 만들 모델의 아키텍처를 이해해보자.  
합성곱 신경망은 출처에 따라 합성곱 층을 부르는 단위가 조금 다르다.  

#### 1) 첫 번째 표기 방법

합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고,  
맥스 풀링(nn.MaxPoold2d)는 풀링 층으로 별도로 명명한다.  

#### 2) 두 번째 표기 방법

합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스 풀링(nn.MaxPoold2d)를  
하나의 합성곱 층으로 본다.  

본 챕터에는 편의상 맥스풀링도 포함하는 것으로 한다.

### 2. 모델 구현하기

#### 1) 필요한 도구 임포트, 입력의 정의

In [1]:
import torch
import torch.nn as nn

In [2]:
# 배치 크기 * 채널 * 높이(height) * 너비(width)의 크기의 텐서를 선언
inputs = torch.Tensor(1,1,28,28)
print(f'텐서의 크기 : {inputs.shape}')

텐서의 크기 : torch.Size([1, 1, 28, 28])


#### 2) Convolution layer(합성곱층)과 Pooling

In [4]:
# 1채널, 32채널 출력, 커널 사이즈 3, 패딩 1
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
# 32채널 입력, 64채널 출력, 커널 사이즈 3, 패딩 1
conv2 = nn.Conv2d(32, 64, kernel_size=3,padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


맥스풀링을 구현하자.  
정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당값으로 지정된다.

In [6]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


#### 3) 구현체를 연결하여 모델 만들기

이제 이들을 연결시켜서 모델을 완성시켜보자.  
입력을 첫 번째 합성곱층을 통과시키자.  

In [7]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


32채널의 28 X 28 텐서가 되었다.  
32채널은 conv1의 out_channel이 32로 설정했기 때문.  
크기가 28인 이유는 feature map 크기 계산 공식에 따라서 나옴.  

이제 맥스풀링에 통과시키고 텐서의 크기를 보자.

In [8]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


stride가 2였기 때문에 28x28이 14x14가 됨.

이제 다시 두 번째 합성곱층에 통과시켜보자.

In [9]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


64가 나온 이유는 conv2의 out_chaannel을 64를 지정했기 때문이다.  
14 x 14가 된 이유는 마찬가지로 feature map 공식.

이제 다시 맥스풀링에 통과시켜 보자.  
역시 stride 2이기 때문에 절반이 되었다.

In [10]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


이제 이 텐서를 펼치는 작업을 하자.  
이에 앞서 텐서의 n번째 차원에 접근하게 해주는 .size(n)에 대해서 배우자.  
현재 out의 크기는 [1, 64, 7, 7]이다.  
out의 첫 번째 차원이 몇인지 출력해보자.  

In [13]:
out.size(0)

1

In [14]:
out.size(1)

64

In [15]:
out.size(2)

7

In [16]:
out.size(3)

7

이제 이를 가지고 .view()를 사용하여 텐서를 펼치는 작업을 해보자.  

In [17]:
out = out.view(out.size(0), -1) # 첫 번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라.
print(out.shape)

torch.Size([1, 3136])


배치 차원을 제외하고 모두 하나의 차원으로 통합된 것을 볼 수 있다.  
이제 이에 대해서 Fully-Connected layer를 통과시켜보자.  

출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환하자.

In [18]:
fc = nn.Linear(3136, 10) # input_dim = 3,136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])
